<a href="https://colab.research.google.com/github/fasthill/ML-DL-study-alone/blob/main/5-1%20%EA%B2%B0%EC%A0%95%20%ED%8A%B8%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Analyse from LightGBM

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/hg-mldl/blob/master/5-1.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩에서 실행하기</a>
  </td>
</table>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from functools import reduce
import datetime
import os, re

In [2]:
from xgboost import XGBClassifier
import xgboost
from lightgbm import LGBMClassifier
import lightgbm
import joblib

In [3]:
from skopt import BayesSearchCV
# parameter ranges are specified by one of below
from skopt.space import Real, Categorical, Integer

In [4]:
from sklearn.linear_model import LogisticRegression, SGDClassifier, SGDRegressor
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [5]:
import pickle

# write list, dictionary to pickle
def save_to_pickle(path, filename):
    open_file = open(path, "wb")
    pickle.dump(filename, open_file)
    open_file.close()

# read list, dictionary from pickle
def load_from_pickle(path):
    open_file = open(path, "rb")
    loaded_file = pickle.load(open_file)
    open_file.close()
    return loaded_file

In [6]:
import csv

# write list, dictionary to csv
# path = './xxx/', my_dict = filename

def save_dict_to_csv(path, my_dict):
    df = pd.DataFrame.from_dict(my_dict, orient='index') 
    df.to_csv (path, index=False, header=True)  
    
def save_list_to_csv(path, my_list):
    df = pd.DataFrame(my_list, columns=['columns'])
    df.to_csv (path, index=False, header=True) 
    
# def load_dict_from_csv(path):
#     df = pd.read_csv(path, header=None)
#     my_dict = df.to_dict()
#     return my_dict

In [7]:
def predict_p(test_target, y_predict_list): 
    ps = precision_score(test_target, y_predict_list)
    rs = recall_score(test_target, y_predict_list)
    fs = f1_score(test_target, y_predict_list)
    roc = roc_auc_score(test_target, y_predict_list)
    collect_list = [ps, rs, fs, roc]
    return collect_list

In [8]:
def make_df_from_estimator(estimator, num):
    df_t = pd.DataFrame.from_dict(estimator, orient='index')
    df_t.columns = [f'value{num}']
    df_t.index.name = 'parameter'
    return df_t

In [9]:
def calc_results(lgbm, model, train_scaled, val_scaled, test_scaled, train_target, val_target, test_target,
                 test_scaled1, test_scaled2, test_scaled3, test_target1, test_target2, test_target3):
# model = lgbmgs.best_estimator_  # 최적의 파라미터로 모델 생성
    y_predict = model.predict(test_scaled)
    result_dict= {}
    result_dict['best_score'] = lgbm.best_score_ 
    result_dict['best_index'] = lgbm.best_index_
    result_dict['acc_train'] = model.score(train_scaled, train_target)
    result_dict['acc_val'] = model.score(val_scaled, val_target)
    result_dict['acc_test'] = model.score(test_scaled, test_target)
    result_dict['precision'] = precision_score(test_target, y_predict)
    result_dict['recall'] = recall_score(test_target, y_predict)
    result_dict['f1score'] = f1_score(test_target, y_predict)
    result_dict['roc'] = roc_auc_score(test_target, y_predict)
    cm = confusion_matrix(test_target, y_predict)
    result_dict['tn'] = cm[0,0]
    result_dict['fp'] = cm[0,1]
    result_dict['fn'] = cm[1,0]
    result_dict['tp'] = cm[1,1]
    #------------------------------------
    y_predict = model.predict(test_scaled1)
    result_dict['acc_test1'] = model.score(test_scaled1, test_target1)
    result_dict['precision1'] = precision_score(test_target1, y_predict)
    result_dict['recall1'] = recall_score(test_target1, y_predict)
    result_dict['f1score1'] = f1_score(test_target1, y_predict)
    result_dict['roc1'] = roc_auc_score(test_target1, y_predict)
    cm = confusion_matrix(test_target1, y_predict)
    result_dict['tn1'] = cm[0,0]
    result_dict['fp1'] = cm[0,1]
    result_dict['fn1'] = cm[1,0]
    result_dict['tp1'] = cm[1,1]
    #------------------------------------
    y_predict = model.predict(test_scaled2)
    result_dict['acc_test2'] = model.score(test_scaled2, test_target2)
    result_dict['precision2'] = precision_score(test_target2, y_predict)
    result_dict['recall2'] = recall_score(test_target2, y_predict)
    result_dict['f1score2'] = f1_score(test_target2, y_predict)
    result_dict['roc2'] = roc_auc_score(test_target2, y_predict)
    cm = confusion_matrix(test_target2, y_predict)
    result_dict['tn2'] = cm[0,0]
    result_dict['fp2'] = cm[0,1]
    result_dict['fn2'] = cm[1,0]
    result_dict['tp2'] = cm[1,1]
    #------------------------------------
    y_predict = model.predict(test_scaled3)
    result_dict['acc_test3'] = model.score(test_scaled3, test_target3)
    result_dict['precision3'] = precision_score(test_target3, y_predict)
    result_dict['recall3'] = recall_score(test_target3, y_predict)
    result_dict['f1score3'] = f1_score(test_target3, y_predict)
    result_dict['roc3'] = roc_auc_score(test_target3, y_predict)
    cm = confusion_matrix(test_target3, y_predict)
    result_dict['tn3'] = cm[0,0]
    result_dict['fp3'] = cm[0,1]
    result_dict['fn3'] = cm[1,0]
    result_dict['tp3'] = cm[1,1]
    return result_dict

In [10]:
# save parameters and columns used for the analysis
def save_parameters(iter, com_name, dt, precision, gsbs, params, new_col):
    save_to_pickle(f'./{com_name}/params_{gsbs}_{dt}_{round(precision*100):2d}%_ver{iter}.pkl', params)
    save_dict_to_csv(f'./{com_name}/params_{gsbs}_{dt}_{round(precision*100):2d}%_ver{iter}.csv', params)
    save_to_pickle(f'./{com_name}/columns_{gsbs}_{dt}_{len(new_col)}_{round(precision*100):2d}%_ver{iter}.pkl', new_col)
    save_list_to_csv(f'./{com_name}/columns_{gsbs}_{dt}_{len(new_col)}_{round(precision*100):2d}%_ver{iter}.csv', new_col)

In [11]:
def save_best_results(com_name, model, scaler, columns, df_base):
    joblib.dump(model, f'./{com_name}/best_model.pkl') # estimaor 저장
    joblib.dump(scaler, f'./{com_name}/best_scaler.pkl') # scaler 저장
    save_to_pickle(f'./{com_name}/best_model_p.pkl', model)
    save_to_pickle(f'./{com_name}/best_scaler_p.pkl', scaler)
    save_to_pickle(f'./{com_name}/best_columns.pkl', new_col)
    save_list_to_csv(f'./{com_name}/best_columns.csv', new_col)
    df_base.to_pickle(f'./{com_name}/best_result.pkl')
    return

In [12]:
code = {'005930' : ['삼성전자', 'sec'], '373220' : ['LG에너지솔루션', 'lgenergy'], 
        '000660' : ['SK하이닉스', 'skhinix'], '207940' : ['삼성바이오로직스', 'ssbio'],
        '006400' : ['삼성SDI', 'sdi'], '051910' : ['LG화학', 'lgchemical'],
        '005935' : ['삼성전자우', 'secpre'], '005380' : ['현대차', 'hyunmotor'],
        '035420' : ['NAVER', 'naver'], '000270' : ['기아','kia'],
        '035720' : ['카카오', 'kakao'], '005490' : ['POSCO홀딩스', 'poscoholding'],
        '105560' : ['KB금융', 'kbbank'], '028260' : ['삼성물산', 'sscnt'],
        '068270' : ['셀트리온', 'celltrion'], '012330' : ['현대모비스', 'mobis'],
        '055550' : ['신한지주', 'shgroup'], '066570' : ['LG전자', 'lgelec'],
        '003670' : ['포스코케미칼', 'poscochemical'], '096770' : ['SK이노베이션', 'skinnovation'],
        '033780' : ['KT&G', 'ktng'], '030200' : ['KT', 'kt']}

code = {'035420' : ['NAVER', 'naver']}

In [13]:
col_inv1 = ['retail_1', 'foreigner_1', 'institution_1', 'financial_1', 'invtrust_1', 'pension_1', 
#             'privequity_1', 'bank_1', 'insurance_1', 'financeetc_1', 'corporateetc_1', 
            'privequity_1',  'insurance_1', 'corporateetc_1', # bank_1, 'financeetc_1 제외
            'foreigneretc_1']
col_inv2 = ['retail_2', 'foreigner_2', 'institution_2', 'financial_2', 'invtrust_2', 'pension_2',
#             'privequity_2', 'bank_2', 'insurance_2', 'financeetc_2', 'corporateetc_2', 
            'privequity_2', 'insurance_2', 'corporateetc_2', # bank_2, 'financeetc_2 제외
            'foreigneretc_2']
col_his1 = ['open_1', 'high_1', 'low_1', 'close_1', 'vol_1']
col_his2 = ['open_2', 'high_2', 'low_2', 'close_2', 'vol_2']
col_cr = ['weekday', 'cr_00', 'cr_05', 'cr_10', 'cr_15', 'cr_20']
col_common1 = ["dji_cr", "dji_f_cr", "dxy_cr", "ixic_f_cr", "bond_kor_10_cr", "bond_kor_2_cr", "kosdaq_cr", "kospi_cr", 
         "krw_cr", "ixic_cr", "spx_f_cr", "sox_cr", "spx_cr", "bond_usa_10_cr", "bond_usa_2_cr", "bond_usa_3m_cr", 
         "vix_cr", "wti_cr", "spsy_cr", "spny_cr", "spxhc_cr", "splrcd_cr", "splrci_cr", "splrcu_cr", "splrcs_cr",
         "splrct_cr", "splrcl_cr", "splrcm_cr", "ixbk_cr", "ixfn_cr", "ixid_cr", "ixis_cr", "ixk_cr", "ixtr_cr",
         "ixut_cr", "nbi_cr", "bkx_cr"]
col_common2 = ["dji_cr_2", "dji_f_cr_2", "dxy_cr_2", "ixic_f_cr_2", "bond_kor_10_cr_2", "bond_kor_2_cr_2", "kosdaq_cr_2", "kospi_cr_2",
         "krw_cr_2", "ixic_cr_2", "spx_f_cr_2", "sox_cr_2", "spx_cr_2", "bond_usa_10_cr_2", "bond_usa_2_cr_2", "bond_usa_3m_cr_2",
         "vix_cr_2", "wti_cr_2", "spsy_cr_2", "spny_cr_2", "spxhc_cr_2", "splrcd_cr_2", "splrci_cr_2", "splrcu_cr_2",
         "splrcs_cr_2", "splrct_cr_2", "splrcl_cr_2", "splrcm_cr_2", "ixbk_cr_2", "ixfn_cr_2", "ixid_cr_2",
         "ixis_cr_2", "ixk_cr_2", "ixtr_cr_2", "ixut_cr_2", "nbi_cr_2", "bkx_cr_2"]
col_futures = ['ixic_f_cr', 'ixic_f_cr_2', 'spx_f_cr', 'spx_f_cr_2', 'dji_f_cr', 'dji_f_cr_2',
           'wti_cr','wti_cr_2', 'dxy_cr', 'dxy_cr_2', 'bond_usa_10_cr', 'bond_usa_10_cr_2' ]
column_o = col_inv1 + col_common1 + col_his1 + col_inv2 + col_common2 + col_his2 + col_cr

col_except_futures = [item for item in column_o if item not in col_futures]

new_col = col_except_futures.copy()

# bank, financeetc는 결측치가 많아서 사용하지 않음.
# df.drop(['bank_1', 'bank_2', 'financeetc_1', 'financeetc_2'], axis=1, inplace=True)   

In [14]:
# 최초의 empty df 생성
df_base = pd.DataFrame(pd.Series([],dtype=pd.StringDtype(), name='parameter')).set_index('parameter')
iter = 0

In [15]:
#### 신규 분석 데이터는 아래 것으로 사용

com_name = list(code.values())[0][1]

directory_for_ml = '../data/data_for_ml/predict/'
fname = f'df_{com_name}_combine.pkl'
f_name = directory_for_ml + fname
df_o = pd.read_pickle(f_name) 
df_o = df_o.iloc[:-1] # /predict/를 사용할 경우 마지막 prediction data 제외

In [16]:
len(df_o)

300

In [107]:
# 반복 작업시 여기서 부터 진행 (feature importance로 선정된 새로운 column으로)

df = df_o[new_col]  # 새롭게 선정된 column으로 진행

# train, val,: 8, test: 2
split_ratio = 0.8
split_n = int(len(df)*split_ratio)

test_interval = int((len(df) - split_n)/3)
data = df.iloc[:split_n, :-5]
target = df.iloc[:split_n, -5]
test_input = df.iloc[split_n:, :-5]
test_target = df.iloc[split_n:, -5]
test_input1 = df.iloc[split_n:split_n+test_interval, :-5]
test_input2 = df.iloc[split_n+test_interval: split_n+test_interval*2, :-5]
test_input3 = df.iloc[split_n+test_interval*2:, :-5]
test_target1 = df.iloc[split_n:split_n+test_interval, -5]
test_target2 = df.iloc[split_n+test_interval: split_n+test_interval*2, -5]
test_target3 = df.iloc[split_n+test_interval*2:, -5]

train_input, val_input, train_target, val_target = \
     train_test_split(data, target, random_state=42, test_size=0.2, stratify=target)

scaler = None
scaler = StandardScaler()
scaler.fit(train_input)
train_scaled = scaler.transform(train_input)
val_scaled = scaler.transform(val_input)
test_scaled = scaler.transform(test_input)
test_scaled1 = scaler.transform(test_input1)
test_scaled2 = scaler.transform(test_input2)
test_scaled3 = scaler.transform(test_input3)

## BayesSearchCV

In [108]:
search_space = {
    'boosting_type' : ['gbdt', 'dart', ],# ['gbdt', 'dart', 'goss'],
    'learning_rate': (0.0001, 0.004, 'log-uniform'),
    'num_leaves': (3, 15),      
    'max_depth': (0, 50), #[-1], #(0, 50),
    'min_child_samples': (5, 40),
    'min_child_weight': (0, 15), # 가장 마지막으로 조정함.  precison이 마지막으로 true positive가 올라감.
    'max_bin': (100, 1000), # 사용시 0.75%로 올라감 default: 255
#     'max_cat_threshold' : (1, 100),
    'subsample': (0.005, 1.0, 'uniform'), # == bagging_fraction
#     'subsample_freq': (1, 10),
    'colsample_bytree': (0.005, 1.0, 'uniform'),
    'subsample_for_bin': (100000, 1000000),
    'scale_pos_weight': np.arange(0.1, 2, 0.05) ,
    'n_estimators': (700, 4000),
#     'reg_alpha' : (0, 4.0, 'uniform'), # =='lambda_l1': [0, 5], # default 0
#     'reg_lambda' : (0, 4.0, 'uniform'), # == lambda_l2': [0, 5], #default 0    
    'reg_alpha' : np.arange(0, 2, 0.1), # 2로 바꾸니 precision이 많이 올라감.
    'reg_lambda' : np.arange(0, 2, 0.1),
    'force_col_wise': ['true'], 
    'boost_from_average' : [True, False],
    'objective': ['binary'],
    'metric': ['binary_logloss'],
}

In [109]:
param_grid = {
    'cv' : 3,
#     'cv' : None,
#     'scoring' : None,
#     'scoring' : 'f1',
    'scoring' : 'precision',
#     'scoring' : 'accuracy',
    'num_col' : len(new_col),
    'iterations' : 30,
             }

1.min child weight 삭제 -> 올라감
2.accuracy로 수정 -> 많이 내려감 따라서 precision으로 하고,
3.n-estimator 4000에서 5000으로
4.'reg_alpha' :(0, 2, 0.1)-=> (0, 3, 0.1), 'reg_lambda' : (0, 2, 0.1),-> (0, 1, 0.1),
5. 2번으로 조정하고, iteration 60으로 -> 변함없음. -> 40으로 조정 변경 변함없음. -> 30으로 같음.
6. subsample_for_bin': (100000, 500000) -> (100000, 1000000) 약간 내려감.
7. 'min_child_weight': (0, 15) 살리고 15로 수정
8. subsample_for_bin': (100000, 1000000) -> (100000, 1100000) , 'reg_alpha' :(0, 2, 0.1)-=> (0, 3, 0.1), 'reg_lambda' : (0, 2, 0.1),-> (0, 3, 0.1) 으로
9. 8을 원상태로 하고, iteration을 30 -> 20으로, cv를 3->5로.
10. 7로 변경

In [110]:
params_space = search_space.copy() # for Bayesian Optimization

In [111]:
# directory가 없으면 만드는 과정
if not os.path.exists(com_name):
    os.makedirs(com_name)
    
while True:
    iter = iter + 1

    lgbm = None
    lgbmBS = None
    gsbs = 'bs'
    
    lgbmBS = BayesSearchCV( estimator = lightgbm.LGBMClassifier(objective='binary',
#                                                                 metric='precision',
                                                                n_jobs=-1,
                                                                verbose=0),
                           search_spaces = params_space,
                           scoring = param_grid['scoring'],
                           cv = StratifiedKFold( n_splits=param_grid['cv'],
                                                 shuffle=True,
                                                 random_state=42 ),
                           n_jobs = -1, # 자동 검색 적용
                           n_iter = param_grid['iterations'],   
                           verbose = 0, refit = True, random_state = 42 
                          )
                          
    print("*** after lgbm BS ******")
    lgbmBS.fit(train_scaled, train_target, eval_metric = 'logloss') 

# save model
    stamp = datetime.datetime.today().isoformat() # 파일명 끝에 생성날짜 시간 추가
    dt = re.sub(r'[-:T]', '', stamp[5:16])
    dt = f'{dt[:4]}_{dt[4:]}'
#     joblib.dump(lgbmBS, f'./{com_name}/estimator_{gsbs}_{dt}_v{iter}.pkl') # bayessearchcv 저장
    save_to_pickle(f'./{com_name}/model_{gsbs}_{dt}_v{iter}.pkl', lgbmBS.best_estimator_)
#     joblib.dump(scaler, f'./{com_name}/scaler_{gsbs}_{dt}_v{iter}.pkl') # scaler 저장
    save_to_pickle(f'./{com_name}/scaler_{gsbs}_{dt}_v{iter}.pkl', scaler)
    
    df_estimator = make_df_from_estimator(lgbmBS.best_estimator_.get_params(), iter)
    result_dict = calc_results(lgbmBS, lgbmBS.best_estimator_, 
                               train_scaled, val_scaled, test_scaled,  
                               train_target, val_target, test_target,
                               test_scaled1, test_scaled2, test_scaled3,  
                               test_target1, test_target2, test_target3
                              )
    
    df_result = make_df_from_estimator(result_dict, iter)
    df_grid = make_df_from_estimator(param_grid, iter)
    df_concat = pd.concat([df_estimator, df_grid, df_result])

    df_base = pd.merge(df_base,df_concat, how='outer', left_index=True, right_index=True)
    
#  4가지 조건이 만족되면 break하고 완료
    val_test = df_concat.loc['acc_val'].iloc[0]
    acc_test = df_concat.loc['acc_test'].iloc[0]
    precision = df_concat.loc['precision'].iloc[0]
    f1score = df_concat.loc['f1score'].iloc[0]
    
    if (val_test >= 0.75 ) & (acc_test > 0.75) & (precision >= 0.8) & (f1score >= 0.6) :
        df_base.to_csv(f'./{com_name}/lgbm_{gsbs}_df_{dt}_{round(precision*100):2d}%_ver{iter}.csv')
        df_base.to_pickle(f'./{com_name}/lgbm_{gsbs}_df_{dt}_{round(precision*100):2d}%_ver{iter}.pkl')
#         params_o = make_new_parameter(params_o, df_concat)
        save_parameters(iter, com_name, dt, precision, gsbs, params_space, new_col)
        break
    if iter >= 1 : 
        df_base.to_csv(f'./{com_name}/lgbm_{gsbs}_df_{dt}_{round(precision*100):2d}%_ver{iter}.csv')
        df_base.to_pickle(f'./{com_name}/lgbm_{gsbs}_df_{dt}_{round(precision*100):2d}%_ver{iter}.pkl')
#         params_o = make_new_parameter(params_o, df_concat)
        save_parameters(iter, com_name, dt, precision, gsbs, params_space, new_col)
        break
    print("******* No.{} BS Process is Done! ********".format(iter))
#     params_o = make_new_parameter(params_o, df_concat)
    save_parameters(iter, com_name, dt, precision, gsbs, params_space, new_col)
    
print("**** End of BayesSearchCV Process ****")

*** after lgbm BS ******
**** End of BayesSearchCV Process ****


In [112]:
df_base.iloc[:-6, 10:].head(60)

,value11,value12,value13,value14,value15,value16,value17,value18
parameter,,,,,,,,
boosting_type,gbdt,gbdt,gbdt,dart,gbdt,dart,gbdt,gbdt
class_weight,None,None,None,None,None,None,None,None
colsample_bytree,0.421697,0.306893,0.167789,0.784319,0.369986,0.784319,0.167789,1.0
importance_type,split,split,split,split,split,split,split,split
learning_rate,0.00031,0.002423,0.001916,0.003807,0.004,0.003807,0.001916,0.003306
max_depth,28,7,23,47,50,47,23,1
min_child_samples,18,27,32,28,40,28,32,27
min_child_weight,0.001,5,8,10,0,10,8,5
min_split_gain,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [102]:
# 최종 가장 좋은 결과 저장
save_best_results(com_name, lgbmBS.best_estimator_, scaler, new_col, df_base)

In [ ]:
print(lgbmBS.best_estimator_)

In [103]:
model = lgbmBS.best_estimator_
feature_df = pd.DataFrame(model.booster_.feature_importance(importance_type='gain'), 
                      index=data.columns, columns=['importance']).sort_values(by='importance', 
                                                                              ascending=False)

In [104]:
len(feature_df)

93

In [105]:
feature_df.tail(40)

,importance
spsy_cr,10.022542
foreigner_1,9.781338
open_2,9.357003
ixtr_cr,9.007152
bkx_cr_2,8.102719
kosdaq_cr,7.463862
kosdaq_cr_2,7.370149
corporateetc_1,6.975944
retail_1,5.555070
sox_cr_2,5.163250


In [106]:
new_col = list(feature_df.index[:84]) +  ['cr_00', 'cr_05', 'cr_10', 'cr_15', 'cr_20']

In [ ]:
new_col

### GridSearchCV

In [ ]:
new_col = column_o.copy()

In [ ]:
# 반복 작업시 여기서 부터 진행 (feature importance로 선정된 새로운 column으로)

df = df_o[new_col]  # 새롭게 선정된 column으로 진행

# train, val,: 8, test: 2
split_ratio = 0.8
split_n = int(len(df)*split_ratio)

test_interval = int((len(df) - split_n)/3)
data = df.iloc[:split_n, :-5]
target = df.iloc[:split_n, -5]
test_input = df.iloc[split_n:, :-5]
test_target = df.iloc[split_n:, -5]
test_input1 = df.iloc[split_n:split_n+test_interval, :-5]
test_input2 = df.iloc[split_n+test_interval: split_n+test_interval*2, :-5]
test_input3 = df.iloc[split_n+test_interval*2:, :-5]
test_target1 = df.iloc[split_n:split_n+test_interval, -5]
test_target2 = df.iloc[split_n+test_interval: split_n+test_interval*2, -5]
test_target3 = df.iloc[split_n+test_interval*2:, -5]

train_input, val_input, train_target, val_target = \
     train_test_split(data, target, random_state=42, test_size=0.2, stratify=target)

scaler = StandardScaler()
scaler.fit(train_input)
train_scaled = scaler.transform(train_input)
val_scaled = scaler.transform(val_input)
test_scaled = scaler.transform(test_input)
test_scaled1 = scaler.transform(test_input1)
test_scaled2 = scaler.transform(test_input2)
test_scaled3 = scaler.transform(test_input3)

In [ ]:
param_grid = {
    'cv' : 5,
#     'cv' : None,
#     'scoring' : None,
    'scoring' : 'precision',
#     'scoring' : 'accuracy',
    'num_col' : len(new_col),
    'iterations' : 10,
             }

In [ ]:
param_base = {'boosting_type' : ['gbdt'], # ['gbdt', dart', 'goss'], # dart : 신경망의 드롭아웃을 적용시킨 방법, 
              'num_leaves' :  [3, 4, 5,], # 두번째로 중요, num_leaves는 작은 데이터면 작은 숫자로
              'learning_rate' :  [0.0003, 0.0005, 0.001, 0.0015,],
#               'max_delta_step' :  [0.4, 0.5, 0.6],
              'n_estimators' :  [ 1100, 1200, 1300, 1400, ],
              'colsample_bytree' :  [0.3, 0.4, 0.5,], # = feature fraction, column sampling. 위의 subsample과 같이 튜닝.
              'subsample' : [0.3, 0.4, ], #  [0.1, 0.2, 0.3, 0.4], # 세번째로 중요. = bagging fraction, row sampling. 아래 colsample_bytree과 같이 튜닝.
              'max_depth' :  [-1], # 가장 먼저 튜닝 필요 -1이 default (무한깊이) 일반적으로 default가 가장 좋음.
              'objective': ['binary'],
              'metric': ['binary_logloss'],
              'scale_pos_weight': [1.0, 1.5,], # posiive  증가, class imbalance 경감, scale_pos_weight > 0.0, default=1.0. 
#               'min_child_samples' : [30],
#               'lambda_l1': [0, 5], # default 0
#               'lambda_l2': [0, 5], #default 0

In [ ]:
params_o = param_base.copy()

In [ ]:
# directory가 없으면 만드는 과정
if not os.path.exists(com_name):
    os.makedirs(com_name)
    
while True:
    iter = iter + 1

    lgbm = None
    lgbmgs = None
    gsbs = 'gs'

    lgbm = lightgbm.LGBMClassifier(random_state=42)

    lgbmgs = GridSearchCV(estimator = lgbm,
                          param_grid = params_o,
#                           cv = 10, # StratifiedKFold us default for binary or multiclass
#                           scoring = 'precision', 
#                           scoring = 'accuracy', 
#                           scoring = ['accuracy', 'precision'], # refit 사용해야 함. 고로 사용하지 않음.
                          cv = param_grid['cv'],
                          scoring = param_grid['scoring'],
                          error_score='raise',
                          verbose = 1,
                          n_jobs=-1, # 자동 검색 적용
                         )
                          
    print("*** after lgbmgs ******")
    lgbmgs.fit(train_scaled, train_target, eval_metric = 'logloss') 
#     lgbmgs.fit(train_scaled, train_target, eval_metric = 'logloss', eval_set = (val_scaled, val_target)) 
    # eval_set가 있어야 "early_stopping_rounds"를 사용할 수 있음.

# save model
    stamp = datetime.datetime.today().isoformat() # 파일명 끝에 생성날짜 시간 추가
    dt = re.sub(r'[-:T]', '', stamp[5:16])
    dt = f'{dt[:4]}_{dt[4:]}'
#     joblib.dump(lgbmBS, f'./{com_name}/estimator_{gsbs}_{dt}_v{iter}.pkl') # bayessearchcv 저장
    save_to_pickle(f'./{com_name}/estimator_{gsbs}_{dt}_v{iter}.pkl', lgbmBS)
#     joblib.dump(scaler, f'./{com_name}/scaler_{gsbs}_{dt}_v{iter}.pkl') # scaler 저장
    save_to_pickle(f'./{com_name}/scaler_{gsbs}_{dt}_v{iter}.pkl', scaler)
    
    df_estimator = make_df_from_estimator(lgbmgs.best_estimator_.get_params(), iter)
    result_dict = calc_results(lgbmgs, lgbmgs.best_estimator_, 
                               train_scaled, val_scaled, test_scaled,  
                               train_target, val_target, test_target,
                               test_scaled1, test_scaled2, test_scaled3,  
                               test_target1, test_target2, test_target3
                              )
    
    df_result = make_df_from_estimator(result_dict, iter)
    df_grid = make_df_from_estimator(param_grid, iter)
    df_concat = pd.concat([df_estimator, df_grid, df_result])

    df_base = pd.merge(df_base,df_concat, how='outer', left_index=True, right_index=True)
    
#  4가지 조건이 만족되면 break하고 완료
    val_test = df_concat.loc['acc_val'].iloc[0]
    acc_test = df_concat.loc['acc_test'].iloc[0]
    precision = df_concat.loc['precision'].iloc[0]
    f1score = df_concat.loc['f1score'].iloc[0]
    
    if (val_test >= 0.75 ) & (acc_test > 0.75) & (precision >= 0.8) & (f1score >= 0.6) :
        df_base.to_csv(f'./{com_name}/lgbm_{gsbs}_df_{dt}_{round(precision*100):2d}%_ver{iter}.csv')
        df_base.to_pickle(f'./{com_name}/lgbm_{gsbs}_df_{dt}_{round(precision*100):2d}%_ver{iter}.pkl')
#         params_o = make_new_parameter(params_o, df_concat)
        save_parameters(iter, com_name, dt, precision, gsbs, params_space, new_col)
        break
    if iter >= 1 : 
        df_base.to_csv(f'./{com_name}/lgbm_{gsbs}_df_{dt}_{round(precision*100):2d}%_ver{iter}.csv')
        df_base.to_pickle(f'./{com_name}/lgbm_{gsbs}_df_{dt}_{round(precision*100):2d}%_ver{iter}.pkl')
#         params_o = make_new_parameter(params_o, df_concat)
        save_parameters(iter, com_name, dt, precision, gsbs, params_space, new_col)
        break
    print("******* No.{}  Process is Done! ********".format(iter))
#     params_o = make_new_parameter(params_o, df_concat)
    save_parameters(iter, com_name, dt, precision, gsbs, params_space, new_col)
    
print("**** End of GridSearCV Process ****")

In [ ]:
df_base.head(60)

In [ ]:
print(lgbmgs.best_estimator_)

In [ ]:
model = lgbmgs.best_estimator_
feature_df = pd.DataFrame(model.booster_.feature_importance(importance_type='gain'), 
                      index=data.columns, columns=['importance']).sort_values(by='importance', 
                                                                              ascending=False)

In [ ]:
len(feature_df)

In [ ]:
feature_df.tail(70)

In [ ]:
new_col = list(feature_df.index[:35]) +  ['cr_00', 'cr_05', 'cr_10', 'cr_15', 'cr_20']